# Topic Modeling (LDA)
## Setup


In [1]:
!pip install --upgrade gensim

In [2]:
!pip install nltk
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.1 MB/s eta 0:00:00


## import libraries

In [3]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import numpy as np
import json
import glob

#gensim
import gensim
import gensim.corpora as copora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Data Prep

In [5]:
def laod_data(file):
  with open (file, "r", encoding="utf-8") as f:
    data = json.load(f)
  return (data)

def write_data(file, data):
  with open (file, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4)

In [6]:
stopwords = stopwords.words("english")
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [38]:
data = laod_data("/content/Conscientiousness_answers_and_comments.json")["comments"]

print(data[0][0:90])

wouldn't setEnabled(true/false) be more appropriate?


In [39]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
  nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
  texts_out = []
  for text in texts:
    doc = nlp(text)
    new_text = []
    for token in doc:
      if token.pos_ in allowed_postags:
        new_text.append(token.lemma_)
    final = " ".join(new_text)
    texts_out.append(final)
  return(texts_out)

lemmatized_text = lemmatization(data)
print(lemmatized_text[0][0:90])

setenabled(true false more appropriate


In [40]:
def gen_words(texts):
  final = []
  for text in texts:
    new = gensim.utils.simple_preprocess(text, deacc=True)
    final.append(new)
  return (final)

data_words = gen_words(lemmatized_text)

print(data_words[0][0:30])

['setenabled', 'true', 'false', 'more', 'appropriate']


In [41]:
id2word = gensim.corpora.Dictionary(data_words)

corpus = []

for text in data_words:
  new = id2word.doc2bow(text) # bow -> bag of words
  corpus.append(new)

print(corpus[0][0:20]) #each tuple contains index of the word along with the frequency of that word.

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]


In [42]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                    id2word=id2word,
                                    num_topics=10,
                                    random_state=100,
                                    update_every=1,
                                    chunksize=100,
                                    passes=10,
                                    alpha="auto")

## Visualization of Data

In [43]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word, mds="mmds", R=9)
vis

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.399910 -0.206037       1        1  24.846719
5     -0.094489  0.370422       2        1  17.056656
7     -0.314595  0.198814       3        1  10.588539
8      0.204819 -0.326347       4        1   9.463929
4      0.339825 -0.090473       5        1   8.983925
1      0.306007  0.144167       6        1   8.700418
2      0.150663  0.300177       7        1   6.815092
9     -0.059551 -0.314276       8        1   5.642727
0     -0.164763 -0.013090       9        1   4.464392
3      0.031994 -0.063357      10        1   3.437606, topic_info=           Term        Freq       Total Category  logprob  loglift
55          use  441.000000  441.000000  Default   9.0000   9.0000
38       answer  236.000000  236.000000  Default   8.0000   8.0000
188        code  151.000000  151.000000  Default   7.0000   7.0000
181        just  203.000000  203.000000  Default   6.0000   6.0000
27     question  139.000000  139.000000  Default   5.0000   5.0000
..          ...         ...         ...      ...      ...      ...
445        year   17.643591   18.468875  Topic10  -3.5939   3.3247
775       clear   17.091804   17.917260  Topic10  -3.6256   3.3232
226  difference   16.186107   17.011428  Topic10  -3.6801   3.3207
481      reason   15.673793   16.499072  Topic10  -3.7122   3.3191
744       local   13.349551   14.175116  Topic10  -3.8727   3.3104

[110 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
127       2  0.075923     add
127       4  0.919510     add
136       2  0.995391    also
38        4  0.995126  answer
1254      3  0.958825     api
...     ...       ...     ...
69        2  0.007732    want
7         1  0.984790    work
7         3  0.007942    work
430       5  0.967349   write
445      10  0.974613    year

[114 rows x 3 columns], R=9, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 6, 8, 9, 5, 2, 3, 10, 1, 4])

In [44]:
print('\nPerplexity: ', lda.log_perplexity(corpus,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.7127234773170095

Coherence Score:  0.42580761670558953
